In [42]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from requests import get, Session, post
import time
from datetime import datetime, timedelta
from io import StringIO
import statistics as stats
import scipy.stats as scistats

In [43]:
uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"



siteCred = {'identity': "space.state@aol.com", 'password': "123SSptaactee123!"}


# use requests package to drive the RESTful session with space-track.org
with Session() as session:
    # run the session in a with block to force session to close if we exit

    # need to log in first. note that we get a 200 to say the web site got the data, not that we are logged in
    resp = session.post(uriBase + requestLogin, data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    print(resp.headers)

    seshcook = resp.headers['Set-Cookie']
    seshcook = seshcook.split(' ')[0][:-1]

{'Date': 'Tue, 19 Nov 2024 07:18:42 GMT', 'Server': 'Apache/2.4.52 (Ubuntu)', 'Set-Cookie': 'chocolatechip=4jq87hhbm9nvpa7vra0eughegdf2g0qn; expires=Tue, 19-Nov-2024 09:18:42 GMT; Max-Age=7200; path=/; HttpOnly', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate', 'Pragma': 'no-cache', 'Status': '200', 'Content-Length': '2', 'Content-Type': 'application/json', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}


In [44]:
spacetrackheaders = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
"accept-encoding": "gzip, deflate, br, zstd",
"accept-language": "en-US,en;q=0.9,es;q=0.8",
"cache-control": "max-age=0",
"cookie": f"spacetrack_csrf_cookie=pc9o9c6722kl32quis48b594gl2cmp5c; {seshcook}",
"priority": "u=0, i",
"sec-ch-ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
"sec-ch-ua-mobile": "?1",
"sec-ch-ua-platform": '"Android"',
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1",
"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36"}

In [45]:
hdrs2 = {"accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
'Authorization': 'TOK:space.state@aol.com',
'accept-encoding':'gzip, deflate, br, zstd',
'accept-language':'en-US,en;q=0.9,es;q=0.8',
'cache-control':'max-age=0',
'cookie':f'spacetrack_csrf_cookie=8u4fvh6rr0rr9bm68ovuf6h2l2v728r2; {seshcook}',
'priority':'u=0, i',
'referer':'https://www.space-track.org/auth/login',
'sec-ch-ua':'"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
'sec-ch-ua-mobile':'?1',
'sec-ch-ua-platform':'"Android"',
'sec-fetch-dest':'document',
'sec-fetch-mode':'navigate',
'sec-fetch-site':'same-origin',
'sec-fetch-user':'?1',
'upgrade-insecure-requests':'1',
'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36"'}

In [46]:
sat_num=40903

In [47]:
today = datetime.today()
today=str(today)
today=today[:10]
today

'2024-11-19'

In [48]:
zr = get(f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/orderby/TLE_LINE1%20ASC/EPOCH/2000-01-01--{today}/format/tle",headers=hdrs2)

In [49]:
print(zr.content)

b'1 40903U 15049E   15263.29071486 -.00000064  00000-0  00000+0 0  9999\r\n2 40903  97.4551 271.4568 0012305 264.7276 219.0254 15.12843012    52\r\n1 40903U 15049E   15263.29071486 -.00000064  00000-0  00000+0 0  9999\r\n2 40903  97.4551 271.4568 0012305 264.7276 219.0254 15.12843012    52\r\n1 40903U 15049E   15263.35872583 -.00000064  00000-0  00000+0 0  9993\r\n2 40903  97.4551 271.5232 0012305 264.4928 229.4290 15.12843012    68\r\n1 40903U 15049E   15263.49685953 -.00000064  00000-0  00000+0 0  9991\r\n2 40903  97.4594 271.6596 0011878 263.4082 262.3431 15.12835531    85\r\n1 40903U 15049E   15263.68321314 -.00000064  00000-0  00000+0 0  9990\r\n2 40903  97.4579 271.8457 0011878 262.7611 197.2753 15.12842365   116\r\n1 40903U 15049E   15263.87966354 -.00000064  00000-0  00000+0 0  9990\r\n2 40903  97.4599 272.0339 0012304 260.2714 189.0099 15.12843257   148\r\n1 40903U 15049E   15263.94100953  .00024761  00000-0  14054-2 0  9999\r\n2 40903  97.4598 272.0931 0012044 258.9248 164.26

In [50]:
oneline = zr.text

In [51]:
tlesraw = oneline.split('\r\n1')

In [52]:
tlesraw

['1 40903U 15049E   15263.29071486 -.00000064  00000-0  00000+0 0  9999\r\n2 40903  97.4551 271.4568 0012305 264.7276 219.0254 15.12843012    52',
 ' 40903U 15049E   15263.29071486 -.00000064  00000-0  00000+0 0  9999\r\n2 40903  97.4551 271.4568 0012305 264.7276 219.0254 15.12843012    52',
 ' 40903U 15049E   15263.35872583 -.00000064  00000-0  00000+0 0  9993\r\n2 40903  97.4551 271.5232 0012305 264.4928 229.4290 15.12843012    68',
 ' 40903U 15049E   15263.49685953 -.00000064  00000-0  00000+0 0  9991\r\n2 40903  97.4594 271.6596 0011878 263.4082 262.3431 15.12835531    85',
 ' 40903U 15049E   15263.68321314 -.00000064  00000-0  00000+0 0  9990\r\n2 40903  97.4579 271.8457 0011878 262.7611 197.2753 15.12842365   116',
 ' 40903U 15049E   15263.87966354 -.00000064  00000-0  00000+0 0  9990\r\n2 40903  97.4599 272.0339 0012304 260.2714 189.0099 15.12843257   148',
 ' 40903U 15049E   15263.94100953  .00024761  00000-0  14054-2 0  9999\r\n2 40903  97.4598 272.0931 0012044 258.9248 164.26

In [53]:
for i in range(len(tlesraw)):
   tlesraw[i] = tlesraw[i].replace('\r\n','|')

In [54]:
for i in range(len(tlesraw)):
    tlesraw[i] = tlesraw[i].replace(' ','|')
    tlesraw[i] = tlesraw[i].split('|')

In [55]:
tle_df = pd.DataFrame(tlesraw)

tle_df['Output']=''

In [56]:
tle_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Output
0,1,40903U,15049E,,,15263.29071486,-.00000064,,00000-0,,...,219.0254,15.12843012,,,,52,None,None,None,
1,,40903U,15049E,,,15263.29071486,-.00000064,,00000-0,,...,219.0254,15.12843012,,,,52,None,None,None,
2,,40903U,15049E,,,15263.35872583,-.00000064,,00000-0,,...,229.4290,15.12843012,,,,68,None,None,None,
3,,40903U,15049E,,,15263.49685953,-.00000064,,00000-0,,...,262.3431,15.12835531,,,,85,None,None,None,
4,,40903U,15049E,,,15263.68321314,-.00000064,,00000-0,,...,197.2753,15.12842365,,,116,None,None,None,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10366,,40903U,15049E,,,23110.71773397,,.02544850,,22897-5,...,249.6724,210.2587,16.26176883427369,None,None,None,None,None,None,
10367,,40903U,15049E,,,23110.71773397,,.02544850,,22897-5,...,249.6724,210.2587,16.26176883427369,None,None,None,None,None,None,
10368,,40903U,15049E,,,23111.38770896,,.02987816,,23123-5,...,251.2590,173.4847,16.30426858427470,None,None,None,None,None,None,
10369,,40903U,15049E,,,23112.43632650,,.18793778,,25107-5,...,175.5276,309.8037,16.51931252427647,None,None,None,None,None,None,


In [57]:
def get_daytime(index_num):
    jday = tle_df.iloc[index_num][5]

    jday = float(jday)

    date, time = divmod(jday, 1.0)

    date = str(int(date))
    #
    #
    #Next convert the fractional day into seconds:
    #

    timedelta(days=time)
    #and add the two together:
    #
    output = datetime.strptime(date, '%y%j') + timedelta(time)

    return output.strftime("TLE,%b %d %Y %H:%M:%S.%f,1 ")

In [58]:
for i in range(len(tle_df[0])):
    tle_df[0].iloc[i] = get_daytime(i)

In [59]:
type(tle_df[27].iloc[0])

NoneType

In [60]:
tle_df = tle_df.map(lambda x: str(x) if type(x) != type(str) else x)
tle_df = tle_df.map(lambda x: '' if x == 'None' else x)

In [61]:
tle_df = tle_df.map(lambda x: x + ' ' if type(x) == str and len(x)>1 else x)

In [62]:
#tle_df = tle_df.map(lambda x: NaN if x == 'None ' else x)

In [63]:

tle_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Output
0,"TLE,Sep 20 2015 06:58:37.763904,1",40903U,15049E,,,15263.29071486,-.00000064,,00000-0,,...,219.0254,15.12843012,,,,52,,,,
1,"TLE,Sep 20 2015 06:58:37.763904,1",40903U,15049E,,,15263.29071486,-.00000064,,00000-0,,...,219.0254,15.12843012,,,,52,,,,
2,"TLE,Sep 20 2015 08:36:33.911712,1",40903U,15049E,,,15263.35872583,-.00000064,,00000-0,,...,229.4290,15.12843012,,,,68,,,,
3,"TLE,Sep 20 2015 11:55:28.663392,1",40903U,15049E,,,15263.49685953,-.00000064,,00000-0,,...,262.3431,15.12835531,,,,85,,,,
4,"TLE,Sep 20 2015 16:23:49.615296,1",40903U,15049E,,,15263.68321314,-.00000064,,00000-0,,...,197.2753,15.12842365,,,116,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10366,"TLE,Apr 20 2023 17:13:32.215008,1",40903U,15049E,,,23110.71773397,,.02544850,,22897-5,...,249.6724,210.2587,16.26176883427369,,,,,,,
10367,"TLE,Apr 20 2023 17:13:32.215008,1",40903U,15049E,,,23110.71773397,,.02544850,,22897-5,...,249.6724,210.2587,16.26176883427369,,,,,,,
10368,"TLE,Apr 21 2023 09:18:18.054144,1",40903U,15049E,,,23111.38770896,,.02987816,,23123-5,...,251.2590,173.4847,16.30426858427470,,,,,,,
10369,"TLE,Apr 22 2023 10:28:18.609600,1",40903U,15049E,,,23112.43632650,,.18793778,,25107-5,...,175.5276,309.8037,16.51931252427647,,,,,,,


In [64]:
type(tle_df[27].iloc[0])

str

In [65]:
tle_df.iloc[0][5]

'15263.29071486 '

In [66]:
jday = tle_df.iloc[0][5]

jday = float(jday)

date, time = divmod(jday, 1.0)

date = str(int(date))
#
#
#Next convert the fractional day into seconds:
#

timedelta(days=time)
#and add the two together:
#
output = datetime.strptime(date, '%y%j') + timedelta(time)

output.strftime("%b %d %Y %H:%M:%S.%f,1")

'Sep 20 2015 06:58:37.763904,1'

In [67]:
for i in tle_df.iloc[10000]:
    print(i)
    print(type(i))

TLE,Jan 16 2023 19:23:49.017408,1  
<class 'str'>
40903U 
<class 'str'>
15049E 
<class 'str'>

<class 'str'>

<class 'str'>
23016.80820622 
<class 'str'>

<class 'str'>
.00095680 
<class 'str'>

<class 'str'>
00000-0 
<class 'str'>

<class 'str'>
83783-3 
<class 'str'>
0
<class 'str'>

<class 'str'>
9995 
<class 'str'>
2
<class 'str'>
40903 
<class 'str'>

<class 'str'>
97.1457 
<class 'str'>

<class 'str'>
81.9983 
<class 'str'>
0007277 
<class 'str'>

<class 'str'>
48.5373 
<class 'str'>

<class 'str'>
88.2178 
<class 'str'>
15.68558397412505 
<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>

<class 'str'>


In [68]:
#tle_df = tle_df.map(lambda x: [] if type(x) == type(None) else x)

In [69]:
for h in range(28):
    for col in tle_df.columns[:-2]:
        for i in range(len(tle_df[col])):
            if tle_df[col].iloc[i] == '' and tle_df[col].iloc[i] != tle_df[col+1].iloc[i]:
                tle_df[col].iloc[i] = tle_df[col+1].iloc[i] 
                tle_df[col+1].iloc[i] = ''


In [70]:
for col in tle_df.columns[:-1]:
    if tle_df[col].iloc[0] == tle_df[col].iloc[len(tle_df)-1] and tle_df[col].iloc[0] == '':
        tle_df = tle_df.drop(columns=col)



In [71]:
tle_df[3] = '  ' + tle_df[3]
tle_df[8] = ''
tle_df[9] = '   1,2'
tle_df[10] = ' ' + tle_df[10] + '  '
tle_df[14] = ' ' + tle_df[14] + ' '


In [72]:
tle_df['Output']=tle_df.sum(axis=1,skipna=True)

In [73]:
tle_df['Output'] = tle_df['Output'] + '\n'

In [74]:
print(tle_df['Output'].iloc[0])

TLE,Sep 20 2015 06:58:37.763904,1  40903U 15049E   15263.29071486 -.00000064 00000-0 00000+0 0   1,2 40903   97.4551 271.4568 0012305  264.7276  219.0254 15.12843012 52 



In [75]:
tle_df['JDATE']=None
for i in range(len(tle_df)):
    tle_df['JDATE'].iloc[i] = tle_df[3].iloc[i]
    tle_df['JDATE'].iloc[i] = tle_df['JDATE'].iloc[i][2:7]

In [76]:
tle_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Output,JDATE
0,"TLE,Sep 20 2015 06:58:37.763904,1",40903U,15049E,15263.29071486,-.00000064,00000-0,00000+0,0,,"1,2",40903,97.4551,271.4568,0012305,264.7276,219.0254,15.12843012,52,"TLE,Sep 20 2015 06:58:37.763904,1 40903U 1504...",15263
1,"TLE,Sep 20 2015 06:58:37.763904,1",40903U,15049E,15263.29071486,-.00000064,00000-0,00000+0,0,,"1,2",40903,97.4551,271.4568,0012305,264.7276,219.0254,15.12843012,52,"TLE,Sep 20 2015 06:58:37.763904,1 40903U 1504...",15263
2,"TLE,Sep 20 2015 08:36:33.911712,1",40903U,15049E,15263.35872583,-.00000064,00000-0,00000+0,0,,"1,2",40903,97.4551,271.5232,0012305,264.4928,229.4290,15.12843012,68,"TLE,Sep 20 2015 08:36:33.911712,1 40903U 1504...",15263
3,"TLE,Sep 20 2015 11:55:28.663392,1",40903U,15049E,15263.49685953,-.00000064,00000-0,00000+0,0,,"1,2",40903,97.4594,271.6596,0011878,263.4082,262.3431,15.12835531,85,"TLE,Sep 20 2015 11:55:28.663392,1 40903U 1504...",15263
4,"TLE,Sep 20 2015 16:23:49.615296,1",40903U,15049E,15263.68321314,-.00000064,00000-0,00000+0,0,,"1,2",40903,97.4579,271.8457,0011878,262.7611,197.2753,15.12842365,116,"TLE,Sep 20 2015 16:23:49.615296,1 40903U 1504...",15263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10366,"TLE,Apr 20 2023 17:13:32.215008,1",40903U,15049E,23110.71773397,.02544850,22897-5,81558-3,0,,"1,2",40903,97.0955,179.6363,0009562,249.6724,210.2587,16.26176883427369,,"TLE,Apr 20 2023 17:13:32.215008,1 40903U 1504...",23110
10367,"TLE,Apr 20 2023 17:13:32.215008,1",40903U,15049E,23110.71773397,.02544850,22897-5,81558-3,0,,"1,2",40903,97.0955,179.6363,0009562,249.6724,210.2587,16.26176883427369,,"TLE,Apr 20 2023 17:13:32.215008,1 40903U 1504...",23110
10368,"TLE,Apr 21 2023 09:18:18.054144,1",40903U,15049E,23111.38770896,.02987816,23123-5,65391-3,0,,"1,2",40903,97.0948,180.3638,0009606,251.2590,173.4847,16.30426858427470,,"TLE,Apr 21 2023 09:18:18.054144,1 40903U 1504...",23111
10369,"TLE,Apr 22 2023 10:28:18.609600,1",40903U,15049E,23112.43632650,.18793778,25107-5,15482-3,0,,"1,2",40903,97.0819,181.5568,0011783,175.5276,309.8037,16.51931252427647,,"TLE,Apr 22 2023 10:28:18.609600,1 40903U 1504...",23112


In [77]:
pwd

'c:\\Users\\Green\\Desktop\\coding projects\\celestrack file\\spoofer'

In [78]:
import os


for i in range(len(tle_df)):
    jate = tle_df['JDATE'].iloc[i]
    filepath = f'DATABASE/StateProcessing/StateDatabase/{jate}_Database_States.txt'

    if os.path.exists(filepath):
        with open(filepath, 'a') as file:
            file.write(tle_df['Output'].iloc[i])
            file.close
    else:
        with open(filepath, 'w') as file:
            file.write(tle_df['Output'].iloc[i])
            file.close
    
    

In [79]:
tle_df.iloc[10]
    

0                       TLE,Sep 21 2015 11:45:16.509024,1  
1                                                   40903U 
2                                                   15049E 
3                                           15264.48977441 
4                                                .00003486 
5                                                  00000-0 
6                                                  20162-3 
7                                                         0
8                                                          
9                                                       1,2
10                                                 40903   
11                                                 97.4577 
12                                                272.6277 
13                                                 0012600 
14                                               259.1380  
15                                                270.8625 
16                                      

In [80]:
tle_df.iloc[10000]

0                       TLE,Jan 16 2023 19:23:49.017408,1  
1                                                   40903U 
2                                                   15049E 
3                                           23016.80820622 
4                                                .00095680 
5                                                  00000-0 
6                                                  83783-3 
7                                                         0
8                                                          
9                                                       1,2
10                                                 40903   
11                                                 97.1457 
12                                                 81.9983 
13                                                 0007277 
14                                                48.5373  
15                                                 88.2178 
16                                      

In [81]:
for i in range(len(tle_df)):
    tle_df[i] = tle_df[i].split(' ')

AttributeError: 'Series' object has no attribute 'split'

In [688]:
len(tle_df)

10371

In [689]:
tle_df

0        [TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903...
1        [TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903...
2        [TLE,Sep, 20, 2015, 08:36:33.911712,1, , 40903...
3        [TLE,Sep, 20, 2015, 11:55:28.663392,1, , 40903...
4        [TLE,Sep, 20, 2015, 16:23:49.615296,1, , 40903...
                               ...                        
10366    [TLE,Apr, 20, 2023, 17:13:32.215008,1, , 40903...
10367    [TLE,Apr, 20, 2023, 17:13:32.215008,1, , 40903...
10368    [TLE,Apr, 21, 2023, 09:18:18.054144,1, , 40903...
10369    [TLE,Apr, 22, 2023, 10:28:18.609600,1, , 40903...
10370    [TLE,Apr, 22, 2023, 10:28:18.609600,1, , 40903...
Name: Output, Length: 10371, dtype: object

In [690]:
tle_df_try

,"TLE,Sep",20,2015,"06:58:37.763904,1",,40903U,15049E,15263.29071486,-.00000064,00000-0,...,271.4568,0012305,264.7276,219.0254,15.12843012,52,None,None,None,


In [691]:
for h in range(len(tle_df)):
    for i in len(tle_df[h]:
        if i == 'None' or i == '':
            del i
        else: tle_df_try.iloc[h][i] = i

SyntaxError: invalid syntax (2128493293.py, line 2)

In [569]:
tle_df_try

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
1,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
2,68,68,68,68,68,68,68,68,68,68,...,68,68,68,68,68,68,68,68,68,68
3,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
4,116,116,116,116,116,116,116,116,116,116,...,116,116,116,116,116,116,116,116,116,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10366,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,...,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369
10367,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,...,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369,16.26176883427369
10368,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,...,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470,16.30426858427470
10369,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,...,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647,16.51931252427647


In [561]:
tle_df[0]

['TLE,Sep',
 '20',
 '2015',
 '06:58:37.763904,1',
 '',
 '40903U',
 '15049E',
 '15263.29071486',
 '-.00000064',
 '00000-0',
 '00000+0',
 '09999',
 '240903',
 '97.4551',
 '271.4568',
 '0012305',
 '264.7276',
 '219.0254',
 '15.12843012',
 '52',
 'None',
 '']

In [544]:
tle_df[0][20]

'None'

In [536]:
tle_df_try = pd.DataFrame(columns=range(30),index=range(len(tle_df)))

In [537]:
for i in range(len(tle_df_try)):
    tle_df_try.iloc[i] = tle_df.iloc[i]

tle_df_try

ValueError: could not broadcast input array from shape (24,) into shape (30,)

In [505]:
for i in range(len(tle_df_try)):
    for h in range(22):
        tle_df_try[i][h] = tle_df[i][h]

KeyError: 22

TypeError: unhashable type: 'list'

Empty DataFrame
Columns: [[TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903U, 15049E, 15263.29071486, -.00000064, 00000-0, 00000+0, 09999, 240903, 97.4551, 271.4568, 0012305, 264.7276, 219.0254, 15.12843012, 52, None, None, None, ], [TLE,Sep, 20, 2015, 06:58:37.763904,1, , 40903U, 15049E, 15263.29071486, -.00000064, 00000-0, 00000+0, 09999, 240903, 97.4551, 271.4568, 0012305, 264.7276, 219.0254, 15.12843012, 52, None, None, None, ], [TLE,Sep, 20, 2015, 08:36:33.911712,1, , 40903U, 15049E, 15263.35872583, -.00000064, 00000-0, 00000+0, 09993, 240903, 97.4551, 271.5232, 0012305, 264.4928, 229.4290, 15.12843012, 68, None, None, None, ], [TLE,Sep, 20, 2015, 11:55:28.663392,1, , 40903U, 15049E, 15263.49685953, -.00000064, 00000-0, 00000+0, 09991, 240903, 97.4594, 271.6596, 0011878, 263.4082, 262.3431, 15.12835531, 85, None, None, None, ], [TLE,Sep, 20, 2015, 16:23:49.615296,1, , 40903U, 15049E, 15263.68321314, -.00000064, 00000-0, 00000+0, 09990, 240903, 97.4579, 271.8457, 0011878, 262.7611, 19